# <div align="center">Detail understanding of SMPL paper</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


Skinned Multi-Person Linear model (SMPL) is a ***skinned vertex-based*** model that accurately represents a wide variety of body shapes in natural human poses. The parameters of the model are learned from data including the ***rest pose template***, ***blend weights***, ***pose-dependent blend shapes***, ***identity-dependent blend shapes***, and a ***regressor from vertices to joint locations***. Pose-dependent blend shapes are a linear function of the elements of the pose rotation matrices. This simple formulation enables training the entire model from a relatively large number of aligned 3D meshes of different people in different poses.

Traditional methods model how vertices are related to an underlying skeleton structure. Basic linear blend skinning (LBS) models are the most widely used, are supported by all game engines, and are efficient to render. Unfortunately they produce unrealistic deformations at joints including the well-known **taffy** and **bowtie** effects.

Model learns blend shapes to correct for the limitations of standard skinning. Different blend shapes for identity, pose, and soft-tissue dynamics are additively combined with a rest template before being transformed by blend skinning. A ***key component*** of this approach is that we formulate the pose blend shapes as a linear function of the elements of the part rotation matrices. This formulation is different from previous methods (Allen et al. 2006; Merry et al. 2006; Wang and Phillips 2002) and makes training and animating with the blend shapes simple. Because the elements of rotation matrices are bounded, so are the resulting deformations, helping our model generalize better.

Our formulation admits an objective function that penalizes the ***pervertex disparities*** between registered meshes and our model, enabling training from data. To learn how people deform with pose, we use 1786 high-resolution 3D scans of different subjects in a wide variety of poses. We align our template mesh to each scan to create a training set. We optimize the ***blend weights***, ***pose-dependent blend shapes***, the ***mean template shape (rest pose)***, and a ***regressor*** from shape to joint locations to minimize the vertex error of the model on the training set. This joint regressor predicts the location
of the joints as a function of the body shape and is critical to animating realistic pose-dependent deformations for any body shape. All parameters are automatically estimated from the aligned scans.

We learn linear models of male and female body shape from the CAESAR dataset (Robinette et al. 2002) (approximately 2000 scans
per gender) using principal component analysis (PCA). We first register a template mesh to each scan and pose normalize the data, which is critical when learning a vertex-based shape model. The resulting principal components become body shape blend shapes.

SMPL can be represented as an Autodesk Filmbox (FBX) file that can be imported into animation systems. We make the SMPL model available for research purposes and provide scripts to drive our model in Maya, Blender, Unreal Engine and Unity.

## <div align="center">Model Formulation</div>
---------------------------------------------------------------------

The model is defined by:

* mean template shape represented by a vector of N concatenated vertices $\overline{T} \in R^{3N}$ in the zero pose, $\theta^{*}$.

* set of blend weights, $\mathcal{W} \in R^{NxK}$ (Fig. 1(a)).

* blend shape function, $B_{S}(\beta): R^{|\beta|} \to R^{3N}$, that takes as input a vector of shape parameters, $\beta$, (Fig. 1(b)) and outputs a blend shape sculpting the subject identity.

* function to predict K joint locations (white dots in Fig. 1), $J(\beta): R^{|\beta|} \to R^{3K}$, as a function of shape parameters, $\beta$.

* and pose-dependent blend shape function, $B_{p}(\theta): R^{|\theta|} \to R^{3N}$, that takes as input a vector of pose parameters, $\theta$, and accounts for the effects of pose-dependent deformations (Fig. 1(c)). 


The corrective blend shapes of these functions are added together in the rest pose as illustrated in
(Fig. 1(c)). Finally, a standard blend skinning function $W$ (linear or dual-quaternion) is applied to rotate the vertices around the estimated joint centers with smoothing defined by the blend weights. 

The result is a model, $M(\beta, \theta; \Phi): R^{|\theta|x|\beta|} \to R^{3N}$, that maps shape and pose parameters to vertices (Fig. 1(d)). Here $\Phi$ represents the learned model parameters.

<img src='_assets/main.png'>

***Figure 1: SMPL model.*** (a) Template mesh with blend weights indicated by color and joints shown in white. (b) With identity-driven blendshape contribution only; vertex and joint locations are linear in shape vector $\beta$. (c) With the addition of of pose blend shapes in preparation for the split pose; note the expansion of the hips. (d) Deformed vertices reposed by dual quaternion skinning for the split pose.

### Blend skinning

The pose of the body is defined by a standard skeletal rig, where $\theta_k \in R^3$  denotes the axis-angle representation of the relative rotation of part k with respect to its parent in the kinematic tree. Our rig has K = 23 joints, hence a pose $\theta = [\theta_0^T, ..., \theta_k^T]$ is defined by $|\theta|$ = 3 x 23 + 3 = 72 parameters; i.e. 3 for each part plus 3 for the root orientation. 

Let $\overline{\theta} = \theta / ||\theta||$ denote the unit norm axis of rotation. Then the axis angle for every joint ***j*** is transformed to a rotation matrix using the ***Rodrigues formula*** 

<div align="center">$exp(\theta_j) = I + \hat{\bar{\theta_j}} sin(||\theta_j||) + \hat{\bar{\theta_j}}^2 cos(||\theta_j||)$</div>

where 

* $\hat{\bar{\theta}}$ is the skew symmetric matrix of the 3-vector $\bar{\theta}$ and 

* $I$ is the $3x3$ identity matrix.

Using this, the ***standard linear blend skinning*** function $W(\bar{T}, J, \theta, \mathcal{W}): R^{3Nx3Kx|\theta|x|\mathcal{W}|} \to R^{3N}$ takes vertices in the rest pose, $\bar{T}$, joint locations, $J$, a pose, $\theta$, and the blend weights, $\mathcal{W}$, and returns the posed vertices. Each vertex $\bar{t_i}$ in $\bar{T}$ is transformed into $\bar{t_i}'$ (both column vectors in homogeneous coordinates) as 

<div align="center">$\bar{t_i}' = \sum_{k=1}^{K} w_{k, i}  G_k^{'}(\theta, J) \bar{t_i}$</div>

<div align="center">$G_k^{'}(\theta, J) = G_k(\theta, J) G_k(\theta^{*}, J)^{-1}$</div>

<div align="center">$G_k(\theta, J) = \prod_{j \in A(k)} \begin{bmatrix}
       exp(\theta_j) & j_j     \\[0.5em]
       \overrightarrow{0} & 1     \\[0.5em]
     \end{bmatrix}$</div>

where 

* $w_{k,i}$ is an element of the blend weight matrix $\mathcal{W}$, representing how much the rotation of part $k$ effects the vertex $i$

* $exp(\theta_j)$ is the local $3x3$ rotation matrix corresponding to joint $j$

* $G_k(\theta, J)$ is the world transformation of joint k, and 

* $G_k'(\theta, J)$ is the same transformation after removing the transformation due to the rest pose, $\theta^*$. 

* Each $3$-element vector in J corresponding to a single joint center, j, is denoted $j_j$ . 

* $A(k)$ denotes the ordered set of joint ancestors of joint k. 

Note, for compatibility with existing rendering engines, we assume $\mathcal{W}$ is sparse and allow at most four parts to influence a vertex.

SMPL keeps the basic skinning function and instead modify the template in an additive way and learn a function to predict joint locations. Our model, $M(\beta, \theta)$ is then defined by

<div align="center">$M(\beta,\theta) = W(T_P(\beta,\theta), J(\beta), \theta, \mathcal{W})$</div>

<div align="center">$T_P(\beta,\theta) = \bar{T} + B_S(\beta) + B_P(\theta)$</div>

where 

* $B_S(\beta)$ and $B_P(\theta)$ are vectors of vertices representing offsets from the template. 

We refer to these as ***shape and pose blend shapes*** respectively. Given this definition, a vertex $\bar{t_i}$ is transformed according to 

<div align="center">$\bar{t_i}' = \sum_{k=1}^{K} w_{k,i} G_k^{'}(\theta, J(\beta)) (\bar{t_i} + b_{S, i}(\beta) + b_{P, i}(\theta))$</div>

where

* $b_{S,i}(\beta)$, $b_{P,i}(\theta)$ are vertices in $B_S(\beta)$ and $B_P(\theta)$ respectively and represent the shape and pose blend shape offsets for the
vertex $\bar{t_i}$

Hence, the joint centers are now a function of body shape and the template mesh that is deformed by blend skinning is now a
function of both pose and shape. Below we describe each term in detail.